In [1]:
from datetime import timedelta

from pyarrow import parquet
import pandas

In [2]:
get_data = lambda domain: parquet.read_table(f'{domain}.parquet').to_pandas()

# Question 1
Count the number of executions within the executions.parquet file, determine the unique number of `Venue's` and the `date of executions`. Log output this information.

In [3]:
%%time
executions_data = get_data('executions')
executions_data['TradeDate'] = pandas.to_datetime(executions_data['TradeTime']).dt.date
unique_venues = executions_data.groupby(['Venue', 'TradeDate']).nunique().reset_index()

print(f'Total rows: {len(executions_data)}\nUnique Venues by trading date:\n{unique_venues[["Venue", "TradeDate"]]}')

Total rows: 4203
Unique Venues by trading date:
  Venue   TradeDate
0  XBRU  2022-09-02
1  XCSE  2022-09-02
2  XETA  2022-09-02
3  XETB  2022-09-02
4  XETS  2022-09-02
5  XSWX  2022-09-02
CPU times: user 52.8 ms, sys: 4.08 ms, total: 56.9 ms
Wall time: 18.2 ms


# Question 2 
Data Cleaning: 
- Filter executions.paraquet for only `CONTINUOUS_TRADING` trades.
- Log output the # of executions.

In [4]:
%%time
executions_data = executions_data.copy()
filtered_data = executions_data.loc[executions_data['Phase'] == 'CONTINUOUS_TRADING']
print(f'Number of executions in the CONTINUOUS_TRADING state: {len(filtered_data)}')

Number of executions in the CONTINUOUS_TRADING state: 4103
CPU times: user 5.53 ms, sys: 1.15 ms, total: 6.68 ms
Wall time: 1.84 ms


# Question 3
Data Transformation:
- Add column `side`, if quantity is negative, side = 2, if quantity is positive side = 1.
- Complement the data with refdata.parquet
  - Add the primary ticker `primary_ticker`
  - Add the primary mic `primary_mic`

--- 

Adding the column `side`

In [5]:
%%time
executions_data = filtered_data.copy()
executions_data['side'] = 2
executions_data.loc[
    (executions_data['Quantity'] >= 0), 'side'
] = 1
executions_data

CPU times: user 3.11 ms, sys: 202 µs, total: 3.31 ms
Wall time: 955 µs


,ISIN,Currency,Venue,TradeTime,Price,Trade_id,Phase,Quantity,TradeDate,side
100,BE0003851681,EUR,XBRU,2022-09-02 07:39:39.072,91.80,100,CONTINUOUS_TRADING,11,2022-09-02,1
101,BE0003851681,EUR,XBRU,2022-09-02 07:43:05.795,91.90,101,CONTINUOUS_TRADING,16,2022-09-02,1
102,BE0003851681,EUR,XBRU,2022-09-02 07:47:55.688,91.85,102,CONTINUOUS_TRADING,22,2022-09-02,1
103,BE0003851681,EUR,XBRU,2022-09-02 07:50:54.472,91.95,103,CONTINUOUS_TRADING,17,2022-09-02,1
104,BE0003851681,EUR,XBRU,2022-09-02 07:54:06.487,92.05,104,CONTINUOUS_TRADING,23,2022-09-02,1
...,...,...,...,...,...,...,...,...,...,...
4198,DE0005552004,EUR,XETA,2022-09-02 15:29:56.302,36.66,4198,CONTINUOUS_TRADING,37,2022-09-02,1
4199,DE0005552004,EUR,XETA,2022-09-02 15:29:58.824,36.65,4199,CONTINUOUS_TRADING,77,2022-09-02,1
4200,DE0005552004,EUR,XETA,2022-09-02 15:35:26.504,36.60,4200,CONTINUOUS_TRADING,1279,2022-09-02,1
4201,DE000A0LD6E6,EUR,XETA,2022-09-02 15:35:25.884,52.55,4201,CONTINUOUS_TRADING,-35,2022-09-02,2


---
Complementing the data with the reference dataset

In [6]:
%%time
executions_data = executions_data.copy()
reference_data = get_data('refdata')
executions_data = pandas.merge(executions_data, reference_data[['ISIN', 'primary_ticker', 'primary_mic']], on='ISIN', how='left')
executions_data

CPU times: user 4.14 ms, sys: 1.52 ms, total: 5.67 ms
Wall time: 4.35 ms


,ISIN,Currency,Venue,TradeTime,Price,Trade_id,Phase,Quantity,TradeDate,side,primary_ticker,primary_mic
0,BE0003851681,EUR,XBRU,2022-09-02 07:39:39.072,91.80,100,CONTINUOUS_TRADING,11,2022-09-02,1,AED,XBRU
1,BE0003851681,EUR,XBRU,2022-09-02 07:43:05.795,91.90,101,CONTINUOUS_TRADING,16,2022-09-02,1,AED,XBRU
2,BE0003851681,EUR,XBRU,2022-09-02 07:47:55.688,91.85,102,CONTINUOUS_TRADING,22,2022-09-02,1,AED,XBRU
3,BE0003851681,EUR,XBRU,2022-09-02 07:50:54.472,91.95,103,CONTINUOUS_TRADING,17,2022-09-02,1,AED,XBRU
4,BE0003851681,EUR,XBRU,2022-09-02 07:54:06.487,92.05,104,CONTINUOUS_TRADING,23,2022-09-02,1,AED,XBRU
...,...,...,...,...,...,...,...,...,...,...,...,...
4221,DE0005552004,EUR,XETA,2022-09-02 15:29:56.302,36.66,4198,CONTINUOUS_TRADING,37,2022-09-02,1,DHL,XETR
4222,DE0005552004,EUR,XETA,2022-09-02 15:29:58.824,36.65,4199,CONTINUOUS_TRADING,77,2022-09-02,1,DHL,XETR
4223,DE0005552004,EUR,XETA,2022-09-02 15:35:26.504,36.60,4200,CONTINUOUS_TRADING,1279,2022-09-02,1,DHL,XETR
4224,DE000A0LD6E6,EUR,XETA,2022-09-02 15:35:25.884,52.55,4201,CONTINUOUS_TRADING,-35,2022-09-02,2,GXI,XETR


# Question 4
Calculations:
- Best bid price and best ask (bbo) – bbo data in marketdata.parquet
  - Find bbo price at execution, 1 second before execution and 1 second after execution from the marketdata.parquet file and add this data into the final output file – respective column table names `['best_bid','best_ask', 'best_bid_min_1s', 'best_ask_min_1s', 'best_bid_1s','best_ask_1s']`

- Mid-Price – bbo data in marketdata.parquet
  - Find the Mid-Price at execution, 1s before the execution and 1s after the execution – respective column table names `['mid_price', 'mid_price_min_1s', 'mid_price_1s']`

- Calculate Slippage [‘slippage’] at execution price
  - For SELL: `(execution_price – best_bid) / (best_ask – best_bid)`
  - For BUY : `(best_ask – execution_price) / (best_ask – best_bid)`
 
---

### Dislcaimer
I found this section to be the most difficult. I went with the instinct that to find the `bbo price at execution` it would be to match the closest `event_timestamp` in the `marketdata.parquet` from the `TradeTime` in the `execution.parquet`, I have done this by doing a merge sorting by the timestamps.

By creating multiple columns of timedelta +/- 1 second, I felt this was the best way to achieve this, however, from the results, it looks like the price hasn't changed. 

--- 

BBO at execution, 1 second before and 1 second after

In [7]:
%%time
market_data = get_data('marketdata')
q4_executions_data = executions_data.copy()

q4_executions_data['TradeTime'] = pandas.to_datetime(q4_executions_data['TradeTime'])
q4_executions_data['TradeTimeBefore'] = q4_executions_data['TradeTime'] - timedelta(seconds=1)
q4_executions_data['TradeTimeAfter'] = q4_executions_data['TradeTime'] + timedelta(seconds=1)

current = pandas.merge_asof(
    q4_executions_data.sort_values('TradeTime'),
    market_data[['event_timestamp', 'best_bid_price', 'best_ask_price', 'primary_mic']].sort_values('event_timestamp'),
    left_on='TradeTime',
    right_on='event_timestamp',
    direction='nearest',
    by='primary_mic'
)
before = pandas.merge_asof(
    q4_executions_data.sort_values('TradeTimeBefore'),
    market_data[['event_timestamp', 'best_bid_price', 'best_ask_price', 'primary_mic']].sort_values('event_timestamp'),
    left_on='TradeTime',
    right_on='event_timestamp',
    direction='nearest',
    by='primary_mic'
)
after = pandas.merge_asof(
    q4_executions_data.sort_values('TradeTimeAfter'),
    market_data[['event_timestamp', 'best_bid_price', 'best_ask_price', 'primary_mic']].sort_values('event_timestamp'),
    left_on='TradeTime',
    right_on='event_timestamp',
    direction='nearest',
    by='primary_mic'
)

q4_executions_data['best_bid_price'] = current['best_bid_price']
q4_executions_data['best_ask_price'] = current['best_ask_price']
q4_executions_data['best_bid_price_min_1s'] = before['best_bid_price']
q4_executions_data['best_ask_price_min_1s'] = before['best_ask_price']
q4_executions_data['best_bid_1s'] = after['best_bid_price']
q4_executions_data['best_ask_1s'] = after['best_ask_price']
q4_executions_data = q4_executions_data.dropna()
q4_executions_data

CPU times: user 904 ms, sys: 106 ms, total: 1.01 s
Wall time: 913 ms


,ISIN,Currency,Venue,TradeTime,Price,Trade_id,Phase,Quantity,TradeDate,side,primary_ticker,primary_mic,TradeTimeBefore,TradeTimeAfter,best_bid_price,best_ask_price,best_bid_price_min_1s,best_ask_price_min_1s,best_bid_1s,best_ask_1s
0,BE0003851681,EUR,XBRU,2022-09-02 07:39:39.072,91.800,100,CONTINUOUS_TRADING,11,2022-09-02,1,AED,XBRU,2022-09-02 07:39:38.072,2022-09-02 07:39:40.072,44.20,44.50,44.20,44.50,44.20,44.50
1,BE0003851681,EUR,XBRU,2022-09-02 07:43:05.795,91.900,101,CONTINUOUS_TRADING,16,2022-09-02,1,AED,XBRU,2022-09-02 07:43:04.795,2022-09-02 07:43:06.795,19.90,20.00,19.90,20.00,19.90,20.00
2,BE0003851681,EUR,XBRU,2022-09-02 07:47:55.688,91.850,102,CONTINUOUS_TRADING,22,2022-09-02,1,AED,XBRU,2022-09-02 07:47:54.688,2022-09-02 07:47:56.688,19.90,20.00,19.90,20.00,19.90,20.00
3,BE0003851681,EUR,XBRU,2022-09-02 07:50:54.472,91.950,103,CONTINUOUS_TRADING,17,2022-09-02,1,AED,XBRU,2022-09-02 07:50:53.472,2022-09-02 07:50:55.472,19.90,20.00,19.90,20.00,19.90,20.00
4,BE0003851681,EUR,XBRU,2022-09-02 07:54:06.487,92.050,104,CONTINUOUS_TRADING,23,2022-09-02,1,AED,XBRU,2022-09-02 07:54:05.487,2022-09-02 07:54:07.487,22.29,22.42,22.29,22.42,22.29,22.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4211,DE000A0JC8S7,EUR,XETS,2022-09-02 15:36:21.730,62.400,4188,CONTINUOUS_TRADING,-56,2022-09-02,2,D6H,XETR,2022-09-02 15:36:20.730,2022-09-02 15:36:22.730,32.35,32.45,32.35,32.45,32.35,32.45
4217,DE0005552004,EUR,XETA,2022-09-02 15:28:34.017,36.665,4194,CONTINUOUS_TRADING,119,2022-09-02,1,DHL,XETR,2022-09-02 15:28:33.017,2022-09-02 15:28:35.017,31.35,31.36,31.35,31.36,31.35,31.36
4220,DE0005552004,EUR,XETA,2022-09-02 15:29:43.417,36.680,4197,CONTINUOUS_TRADING,70,2022-09-02,1,DHL,XETR,2022-09-02 15:29:42.417,2022-09-02 15:29:44.417,100.70,100.80,100.70,100.80,100.70,100.80
4221,DE0005552004,EUR,XETA,2022-09-02 15:29:56.302,36.660,4198,CONTINUOUS_TRADING,37,2022-09-02,1,DHL,XETR,2022-09-02 15:29:55.302,2022-09-02 15:29:57.302,100.70,100.80,100.70,100.80,100.70,100.80


---

Calculating the `Mid-Price`

In [8]:
%%time
mid_executions_data = q4_executions_data.copy()
mid_executions_data['mid_price'] = mid_executions_data[['best_ask_price', 'best_bid_price']].mean(axis=1)
mid_executions_data['mid_price_min_1s'] = mid_executions_data[['best_ask_price_min_1s', 'best_bid_price_min_1s']].mean(axis=1)
mid_executions_data['mid_price_1s'] = mid_executions_data[['best_ask_1s', 'best_bid_1s']].mean(axis=1)
mid_executions_data

CPU times: user 3.04 ms, sys: 1.33 ms, total: 4.37 ms
Wall time: 3.4 ms


,ISIN,Currency,Venue,TradeTime,Price,Trade_id,Phase,Quantity,TradeDate,side,...,TradeTimeAfter,best_bid_price,best_ask_price,best_bid_price_min_1s,best_ask_price_min_1s,best_bid_1s,best_ask_1s,mid_price,mid_price_min_1s,mid_price_1s
0,BE0003851681,EUR,XBRU,2022-09-02 07:39:39.072,91.800,100,CONTINUOUS_TRADING,11,2022-09-02,1,...,2022-09-02 07:39:40.072,44.20,44.50,44.20,44.50,44.20,44.50,44.350,44.350,44.350
1,BE0003851681,EUR,XBRU,2022-09-02 07:43:05.795,91.900,101,CONTINUOUS_TRADING,16,2022-09-02,1,...,2022-09-02 07:43:06.795,19.90,20.00,19.90,20.00,19.90,20.00,19.950,19.950,19.950
2,BE0003851681,EUR,XBRU,2022-09-02 07:47:55.688,91.850,102,CONTINUOUS_TRADING,22,2022-09-02,1,...,2022-09-02 07:47:56.688,19.90,20.00,19.90,20.00,19.90,20.00,19.950,19.950,19.950
3,BE0003851681,EUR,XBRU,2022-09-02 07:50:54.472,91.950,103,CONTINUOUS_TRADING,17,2022-09-02,1,...,2022-09-02 07:50:55.472,19.90,20.00,19.90,20.00,19.90,20.00,19.950,19.950,19.950
4,BE0003851681,EUR,XBRU,2022-09-02 07:54:06.487,92.050,104,CONTINUOUS_TRADING,23,2022-09-02,1,...,2022-09-02 07:54:07.487,22.29,22.42,22.29,22.42,22.29,22.42,22.355,22.355,22.355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4211,DE000A0JC8S7,EUR,XETS,2022-09-02 15:36:21.730,62.400,4188,CONTINUOUS_TRADING,-56,2022-09-02,2,...,2022-09-02 15:36:22.730,32.35,32.45,32.35,32.45,32.35,32.45,32.400,32.400,32.400
4217,DE0005552004,EUR,XETA,2022-09-02 15:28:34.017,36.665,4194,CONTINUOUS_TRADING,119,2022-09-02,1,...,2022-09-02 15:28:35.017,31.35,31.36,31.35,31.36,31.35,31.36,31.355,31.355,31.355
4220,DE0005552004,EUR,XETA,2022-09-02 15:29:43.417,36.680,4197,CONTINUOUS_TRADING,70,2022-09-02,1,...,2022-09-02 15:29:44.417,100.70,100.80,100.70,100.80,100.70,100.80,100.750,100.750,100.750
4221,DE0005552004,EUR,XETA,2022-09-02 15:29:56.302,36.660,4198,CONTINUOUS_TRADING,37,2022-09-02,1,...,2022-09-02 15:29:57.302,100.70,100.80,100.70,100.80,100.70,100.80,100.750,100.750,100.750


---

### Disclaimer

I couldn't find any data points that explicitly stated whether or not a transaction was a `BUY` or a `SELL`. I'm going off the assumption that if the quantity is negative, this is a `Sell`, and positive being a `Buy`.

---

Calculating slippage:
- For SELL: (execution_price – best_bid) / (best_ask – best_bid)
- For BUY : (best_ask – execution_price) / (best_ask – best_bid)

In [9]:
%%time
def calculate_slippage(row):
    if row.side == 2:
        return (row.Price - row.best_bid_price) / (row.best_ask_price - row.best_bid_price)
    return (row.best_ask_price - row.Price) / (row.best_ask_price - row.best_bid_price)

slippage_data = mid_executions_data.copy()
slippage_data['slippage'] = slippage_data.apply(calculate_slippage, axis=1)

slippage_data

CPU times: user 42.2 ms, sys: 1.72 ms, total: 43.9 ms
Wall time: 43 ms


,ISIN,Currency,Venue,TradeTime,Price,Trade_id,Phase,Quantity,TradeDate,side,...,best_bid_price,best_ask_price,best_bid_price_min_1s,best_ask_price_min_1s,best_bid_1s,best_ask_1s,mid_price,mid_price_min_1s,mid_price_1s,slippage
0,BE0003851681,EUR,XBRU,2022-09-02 07:39:39.072,91.800,100,CONTINUOUS_TRADING,11,2022-09-02,1,...,44.20,44.50,44.20,44.50,44.20,44.50,44.350,44.350,44.350,-157.666667
1,BE0003851681,EUR,XBRU,2022-09-02 07:43:05.795,91.900,101,CONTINUOUS_TRADING,16,2022-09-02,1,...,19.90,20.00,19.90,20.00,19.90,20.00,19.950,19.950,19.950,-719.000000
2,BE0003851681,EUR,XBRU,2022-09-02 07:47:55.688,91.850,102,CONTINUOUS_TRADING,22,2022-09-02,1,...,19.90,20.00,19.90,20.00,19.90,20.00,19.950,19.950,19.950,-718.500000
3,BE0003851681,EUR,XBRU,2022-09-02 07:50:54.472,91.950,103,CONTINUOUS_TRADING,17,2022-09-02,1,...,19.90,20.00,19.90,20.00,19.90,20.00,19.950,19.950,19.950,-719.500000
4,BE0003851681,EUR,XBRU,2022-09-02 07:54:06.487,92.050,104,CONTINUOUS_TRADING,23,2022-09-02,1,...,22.29,22.42,22.29,22.42,22.29,22.42,22.355,22.355,22.355,-535.615385
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4211,DE000A0JC8S7,EUR,XETS,2022-09-02 15:36:21.730,62.400,4188,CONTINUOUS_TRADING,-56,2022-09-02,2,...,32.35,32.45,32.35,32.45,32.35,32.45,32.400,32.400,32.400,300.500000
4217,DE0005552004,EUR,XETA,2022-09-02 15:28:34.017,36.665,4194,CONTINUOUS_TRADING,119,2022-09-02,1,...,31.35,31.36,31.35,31.36,31.35,31.36,31.355,31.355,31.355,-530.500000
4220,DE0005552004,EUR,XETA,2022-09-02 15:29:43.417,36.680,4197,CONTINUOUS_TRADING,70,2022-09-02,1,...,100.70,100.80,100.70,100.80,100.70,100.80,100.750,100.750,100.750,641.200000
4221,DE0005552004,EUR,XETA,2022-09-02 15:29:56.302,36.660,4198,CONTINUOUS_TRADING,37,2022-09-02,1,...,100.70,100.80,100.70,100.80,100.70,100.80,100.750,100.750,100.750,641.400000
